<a href="https://colab.research.google.com/github/PhuphaB/Modelling_Individual_Summative_Assessment/blob/main/Individual_Summative_Assessment_EFIMM0142_24_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 (complete)


##Packages

In [ ]:
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 48.7 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking 

##1a

In [ ]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd
# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimisation problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Objective_Value"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nObjective Value:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       162.0       42.0
9    Feb       C       201.0      113.0
10   Mar       C       126.0       99.0
11   Apr       C        51.0        0.0

Objective Value: 26127.0


##1b

In [ ]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs with increased cost for Product B in March
production_cost = {
    'A': [20, 20, 20, 20],
    'B': [30, 30, 35, 30],  # Increased cost for Product B in March
    'C': [10, 10, 10, 10]
}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p][i] * x[f"{p}_{m}"] for p in products for i, m in enumerate(months)) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "OIbjective_Value"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nObjective Value:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B       190.0      100.0
6    Mar       B         0.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       179.0       59.0
9    Feb       C        71.0        0.0
10   Mar       C       239.0       99.0
11   Apr       C        51.0        0.0

Objective Value: 26279.0


##1c

In [ ]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "objective_Value"

# Define demand constraints with increased demand for Product C in April
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 170]  # Increased demand for Product C in April by 20 units
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nObjective Value:", value(prob.objective))



Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       182.0       62.0
9    Feb       C       201.0      133.0
10   Mar       C       126.0      119.0
11   Apr       C        51.0        0.0

Objective Value: 26357.0


##1d

In [ ]:
import pulp
from pulp import LpVariable, LpProblem, LpStatus, lpSum, LpMinimize, value
import pandas as pd

# Define months and products
months = ['Jan', 'Feb', 'Mar', 'Apr']
products = ['A', 'B', 'C']

# Create decision variables for production and inventory
x = {f"{p}_{m}": LpVariable(f"x_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}
I = {f"{p}_{m}": LpVariable(f"I_{p}_{m}", lowBound=0, cat='Integer') for p in products for m in months}

# Initialize the problem as a minimization problem
prob = LpProblem("Production_Inventory_Optimization", sense=LpMinimize)

# Define costs
production_cost = {'A': 20, 'B': 30, 'C': 10}
holding_cost = {'A': 1, 'B': 2, 'C': 0.5}

# Objective function (Production cost + Holding cost)
prob += (
    lpSum(production_cost[p] * x[f"{p}_{m}"] for p in products for m in months) +
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months)
), "Objective_Value"

# Define demand constraints
demand = {
    'A': [100, 110, 120, 130],
    'B': [80, 90, 100, 110],
    'C': [120, 130, 140, 150]
}
for p in products:
    for i, m in enumerate(months):
        if i == 0:
            prob += (x[f"{p}_{m}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")
        else:
            prob += (x[f"{p}_{m}"] + I[f"{p}_{months[i-1]}"] - I[f"{p}_{m}"] == demand[p][i], f"Demand_{p}_{m}")

# Define production hour constraints
hours_per_unit = {'A': 1.2, 'B': 1.3, 'C': 1}
available_hours = [500, 450, 400, 350]
for i, m in enumerate(months):
    prob += (
        lpSum(hours_per_unit[p] * x[f"{p}_{m}"] for p in products) <= available_hours[i],
        f"Hours_{m}"
    )

# Add constraint for total inventory holding cost not exceeding £500
prob += (
    lpSum(holding_cost[p] * I[f"{p}_{m}"] for p in products for m in months) <= 500,
    "Total_Holding_Cost"
)

# Solve the problem
prob.solve()

# Prepare the results in a structured format
results = []
for p in products:
    for m in months:
        results.append({
            'Month': m,
            'Product': p,
            'Production': x[f"{p}_{m}"].varValue,
            'Inventory': I[f"{p}_{m}"].varValue
        })

# Convert results to a DataFrame for display
results_df = pd.DataFrame(results)

# Print the results
print("\nSolution Status:", LpStatus[prob.status])
print(results_df)
print("\nObjective Value:", value(prob.objective))


Solution Status: Optimal
   Month Product  Production  Inventory
0    Jan       A       100.0        0.0
1    Feb       A       110.0        0.0
2    Mar       A       120.0        0.0
3    Apr       A       130.0        0.0
4    Jan       B        80.0        0.0
5    Feb       B        90.0        0.0
6    Mar       B       100.0        0.0
7    Apr       B       110.0        0.0
8    Jan       C       162.0       42.0
9    Feb       C       201.0      113.0
10   Mar       C       126.0       99.0
11   Apr       C        51.0        0.0

Objective Value: 26127.0


#Question 2 (complete)

##2a

In [ ]:
from pulp import *

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = 500000

# Define the optimization problem
model = LpProblem("Minimize_Total_Cost", LpMinimize)

# Decision variables
x = {
    (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
    for facility in ["Cincinnati", "Oakland"]
    for location in locations
}

# Objective function: Minimize total cost
model += lpSum(
    (production_cost[facility] + (freight_cost[facility][i] if freight_cost[facility][i] is not None else float('inf')))
    * x[(facility, location)]
    for facility in ["Cincinnati", "Oakland"]
    for i, location in enumerate(locations)
    if freight_cost[facility][i] is not None
)

# Constraints: Demand must be satisfied for each location
for i, location in enumerate(locations):
    model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

# Capacity constraint for Cincinnati
model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

# Capacity constraint for Oakland
model += lpSum(x[("Oakland", location)] for location in locations) <= max_oakland

# Solve the problem
model.solve()

# Extract the results
allocation = [
    {
        "Location": location,
        "From Cincinnati": x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0,
        "From Oakland": x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0,
        "Total Gallons": x[("Cincinnati", location)].varValue + x[("Oakland", location)].varValue,
        "Cost": (x[("Cincinnati", location)].varValue * (production_cost["Cincinnati"] + (freight_cost["Cincinnati"][i] if freight_cost["Cincinnati"][i] is not None else float('inf')))) +
                (x[("Oakland", location)].varValue * (production_cost["Oakland"] + (freight_cost["Oakland"][i] if freight_cost["Oakland"][i] is not None else float('inf'))))
    }
    for i, location in enumerate(locations)
]

total_gallons_cincinnati = sum(alloc['From Cincinnati'] for alloc in allocation)
total_gallons_oakland = sum(alloc['From Oakland'] for alloc in allocation)
total_gallons = total_gallons_cincinnati + total_gallons_oakland
total_cost = sum(alloc['Cost'] for alloc in allocation)

# Display the results
print("Results:")
print(f"{'Location':<15} {'From Cincinnati':<20} {'From Oakland':<15} {'Total Gallons':<15} {'Cost':<10}")
for alloc in allocation:
    print(f"{alloc['Location']:<15} {alloc['From Cincinnati']:<20.1f} {alloc['From Oakland']:<15.1f} {alloc['Total Gallons']:<15.1f} {alloc['Cost']:<10.2f}")

print(f"\n{'Total':<15} {total_gallons_cincinnati:<20.1f} {total_gallons_oakland:<15.1f} {total_gallons:<15.1f} {total_cost:<10.2f}")

Results:
Location        From Cincinnati      From Oakland    Total Gallons   Cost      
Santa Ana       0.0                  22418.0         22418.0         41921.66  
El Paso         6800.0               0.0             6800.0          13872.00  
Pendleton       39636.0              40654.0         80290.0         167460.64 
Houston         100447.0             0.0             100447.0        165737.55 
Kansas City     24570.0              0.0             24570.0         38329.20  
Los Angeles     0.0                  64761.0         64761.0         121103.07 
Glendale        0.0                  33689.0         33689.0         62998.43  
Jacksonville    68486.0              0.0             68486.0         105468.44 
Little Rock     148586.0             0.0             148586.0        228822.44 
Bridgeport      111475.0             0.0             111475.0        171671.50 
Sacramento      0.0                  112000.0        112000.0        201600.00 

Total           500000.0      

##2b

In [ ]:
from pulp import *

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = 500000

# Define the optimization problem
model = LpProblem("Minimize_Total_Cost", LpMinimize)

# Decision variables
x = {
    (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
    for facility in ["Cincinnati", "Oakland"]
    for location in locations
}

# Objective function: Minimize total cost
model += lpSum(
    (production_cost[facility] + (freight_cost[facility][i] if freight_cost[facility][i] is not None else float('inf')))
    * x[(facility, location)]
    for facility in ["Cincinnati", "Oakland"]
    for i, location in enumerate(locations)
    if freight_cost[facility][i] is not None
)

# Constraints: Demand must be satisfied for each location
for i, location in enumerate(locations):
    model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

# Capacity constraint for Cincinnati
model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

# Capacity constraint for Oakland
model += lpSum(x[("Oakland", location)] for location in locations) <= max_oakland

# Solve the problem
model.solve()

# Extract the results
allocation = [
    {
        "Location": location,
        "From Cincinnati": x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0,
        "From Oakland": x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0,
        "Total Gallons": x[("Cincinnati", location)].varValue + x[("Oakland", location)].varValue,
        "Cost": (x[("Cincinnati", location)].varValue * (production_cost["Cincinnati"] + (freight_cost["Cincinnati"][i] if freight_cost["Cincinnati"][i] is not None else float('inf')))) +
                (x[("Oakland", location)].varValue * (production_cost["Oakland"] + (freight_cost["Oakland"][i] if freight_cost["Oakland"][i] is not None else float('inf'))))
    }
    for i, location in enumerate(locations)
]

total_gallons_cincinnati = sum(alloc['From Cincinnati'] for alloc in allocation)
total_gallons_oakland = sum(alloc['From Oakland'] for alloc in allocation)
total_gallons = total_gallons_cincinnati + total_gallons_oakland
total_cost = sum(alloc['Cost'] for alloc in allocation)

# Calculate the breakeven point (lowest bid without losing money)
breakeven_price_per_gallon = total_cost / total_gallons

# Display the results
print("Results:")
print(f"{'Location':<15} {'From Cincinnati':<20} {'From Oakland':<15} {'Total Gallons':<15} {'Cost':<10}")
for alloc in allocation:
    print(f"{alloc['Location']:<15} {alloc['From Cincinnati']:<20.1f} {alloc['From Oakland']:<15.1f} {alloc['Total Gallons']:<15.1f} {alloc['Cost']:<10.2f}")

print(f"\n{'Total':<15} {total_gallons_cincinnati:<20.1f} {total_gallons_oakland:<15.1f} {total_gallons:<15.1f} {total_cost:<10.2f}")
print(f"The breakeven price per gallon for Solutions Plus is {breakeven_price_per_gallon:.2f} USD.")



Results:
Location        From Cincinnati      From Oakland    Total Gallons   Cost      
Santa Ana       0.0                  22418.0         22418.0         41921.66  
El Paso         6800.0               0.0             6800.0          13872.00  
Pendleton       39636.0              40654.0         80290.0         167460.64 
Houston         100447.0             0.0             100447.0        165737.55 
Kansas City     24570.0              0.0             24570.0         38329.20  
Los Angeles     0.0                  64761.0         64761.0         121103.07 
Glendale        0.0                  33689.0         33689.0         62998.43  
Jacksonville    68486.0              0.0             68486.0         105468.44 
Little Rock     148586.0             0.0             148586.0        228822.44 
Bridgeport      111475.0             0.0             111475.0        171671.50 
Sacramento      0.0                  112000.0        112000.0        201600.00 

Total           500000.0      

In [ ]:
# Calculate the breakeven point (lowest bid without losing money)
breakeven_price_per_gallon = total_cost / total_gallons
print(f"The breakeven price per gallon for Solutions Plus is {breakeven_price_per_gallon:.2f} USD.")

The breakeven price per gallon for Solutions Plus is 1.71 USD.


##2c

In [ ]:
# Calculate the bid with a standard 15% markup
bid_with_markup_per_gallon = (total_cost / total_gallons) * 1.15

# Display the results
print(f"Bid with Standard 15% Markup per Gallon: ${bid_with_markup_per_gallon:.2f}")

Bid with Standard 15% Markup per Gallon: $1.96


##2d

In [ ]:
# Define the multipliers for different scenarios
multipliers = [1.3, 1.2, 1.1, 1.0, 0.9, 0.8, 0.7]

# Solve the problem for each scenario and print the results
for multiplier in multipliers:
    results, total_cost = solve_problem(multiplier)
    print(f"Results with freight cost multiplier {multiplier}:")
    print(f"Total Cost = {total_cost}\n")

Results with freight cost multiplier 1.3:
Total Cost = 1399287.019

Results with freight cost multiplier 1.2:
Total Cost = 1372519.656

Results with freight cost multiplier 1.1:
Total Cost = 1345752.293

Results with freight cost multiplier 1.0:
Total Cost = 1318984.93

Results with freight cost multiplier 0.9:
Total Cost = 1292217.5669999998

Results with freight cost multiplier 0.8:
Total Cost = 1265450.204

Results with freight cost multiplier 0.7:
Total Cost = 1238682.841



In [ ]:
# Import necessary libraries
from pulp import *
import pandas as pd
from tabulate import tabulate

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000],
    "Oakland": [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = 500000

# Function to solve the optimization problem with given freight cost multiplier
def solve_problem(freight_multiplier):
    # Adjust freight costs based on the multiplier
    adjusted_freight_cost = {
        key: [cost * freight_multiplier for cost in costs]
        for key, costs in freight_cost.items()
    }

    # Initialize the problem
    prob = LpProblem("Minimize_Cost", LpMinimize)

    # Decision variables
    x = LpVariable.dicts("shipment", [(i, j) for i in production_cost.keys() for j in locations], 0, None, LpContinuous)

    # Objective function
    prob += lpSum([production_cost[i] * x[(i, j)] + adjusted_freight_cost[i][k] * x[(i, j)] for i in production_cost.keys() for k, j in enumerate(locations)])

    # Constraints
    # Demand constraints
    for j, loc in enumerate(locations):
        prob += lpSum([x[(i, loc)] for i in production_cost.keys()]) == demand[j]

    # Capacity constraints
    prob += lpSum([x[("Cincinnati", j)] for j in locations]) <= max_cincinnati
    prob += lpSum([x[("Oakland", j)] for j in locations]) <= max_oakland

    # Solve the problem
    prob.solve()

    # Collect the results
    results = []
    for v in prob.variables():
        results.append([v.name, v.varValue])

    total_cost = value(prob.objective)

    return results, total_cost

# Define the multipliers for different scenarios
multipliers = [1.3, 1.2, 1.1, 1.0, 0.9, 0.8, 0.7]

# Run the optimization for each multiplier and store the results
all_results = []
for multiplier in multipliers:
    results, total_cost = solve_problem(multiplier)
    all_results.append({
        "Multiplier": multiplier,
        "Total Cost": total_cost,
        "Results": results
    })

# Display the results
for result in all_results:
    print(f"Multiplier: {result['Multiplier']}")
    print(f"Total Cost: {result['Total Cost']}")
    print(tabulate(result['Results'], headers=["Variable", "Value"]))
    print("\n")

Multiplier: 1.3
Total Cost: 1399287.019
Variable                                   Value
---------------------------------------  -------
shipment_('Cincinnati',_'Bridgeport')     111475
shipment_('Cincinnati',_'El_Paso')          6800
shipment_('Cincinnati',_'Glendale')            0
shipment_('Cincinnati',_'Houston')        100447
shipment_('Cincinnati',_'Jacksonville')    68486
shipment_('Cincinnati',_'Kansas_City')     24570
shipment_('Cincinnati',_'Little_Rock')    148586
shipment_('Cincinnati',_'Los_Angeles')         0
shipment_('Cincinnati',_'Pendleton')       39636
shipment_('Cincinnati',_'Sacramento')          0
shipment_('Cincinnati',_'Santa_Ana')           0
shipment_('Oakland',_'Bridgeport')             0
shipment_('Oakland',_'El_Paso')                0
shipment_('Oakland',_'Glendale')           33689
shipment_('Oakland',_'Houston')                0
shipment_('Oakland',_'Jacksonville')           0
shipment_('Oakland',_'Kansas_City')            0
shipment_('Oakland',_'Little_

##2e

In [ ]:
from pulp import *
import pandas as pd
from tabulate import tabulate

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = 500000

def solve_optimization(production_cost, freight_cost):
    # Define the optimization problem
    model = LpProblem("Minimize_Total_Cost", LpMinimize)

    # Decision variables
    x = {
        (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
        for facility in ["Cincinnati", "Oakland"]
        for location in locations
    }

    # Objective function: Minimize total cost
    model += lpSum(
        (production_cost[facility] + (freight_cost[facility][i] if freight_cost[facility][i] is not None else float('inf')))
        * x[(facility, location)]
        for facility in ["Cincinnati", "Oakland"]
        for i, location in enumerate(locations)
        if freight_cost[facility][i] is not None
    )

    # Constraints: Demand must be satisfied for each location
    for i, location in enumerate(locations):
        model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

    # Capacity constraint for Cincinnati
    model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

    # Capacity constraint for Oakland
    model += lpSum(x[("Oakland", location)] for location in locations) <= max_oakland

    # Solve the problem
    model.solve()

    # Extract the results
    total_cost = value(model.objective)
    shipping_strategy = {(facility, location): x[(facility, location)].varValue for facility in ["Cincinnati", "Oakland"] for location in locations}

    return total_cost, shipping_strategy

# Original scenario
original_total_cost, original_shipping_strategy = solve_optimization(production_cost, freight_cost)

# Scenario with increased costs by 10%
increased_production_cost = {k: v * 1.10 for k, v in production_cost.items()}
increased_freight_cost = {k: [v * 1.10 if v != float('inf') else v for v in vals] for k, vals in freight_cost.items()}
increased_total_cost, increased_shipping_strategy = solve_optimization(increased_production_cost, increased_freight_cost)

# Scenario with decreased costs by 10%
decreased_production_cost = {k: v * 0.90 for k, v in production_cost.items()}
decreased_freight_cost = {k: [v * 0.90 if v != float('inf') else v for v in vals] for k, vals in freight_cost.items()}
decreased_total_cost, decreased_shipping_strategy = solve_optimization(decreased_production_cost, decreased_freight_cost)

# Collect results into a list of dictionaries for better presentation
sensitivity_results = [
    {
        "Scenario": "Original",
        "Total Cost": original_total_cost,
        "Allocation": original_shipping_strategy
    },
    {
        "Scenario": "Increased Costs by 10%",
        "Total Cost": increased_total_cost,
        "Allocation": increased_shipping_strategy
    },
    {
        "Scenario": "Decreased Costs by 10%",
        "Total Cost": decreased_total_cost,
        "Allocation": decreased_shipping_strategy
    }
]

# Display detailed results for each scenario in a more tidy format using tabulate and include scenario number
for idx, result in enumerate(sensitivity_results):
    print(f"Scenario {idx + 1}: {result['Scenario']}")
    print(f"Total Cost: ${result['Total Cost']:.2f}")

    allocation_df = pd.DataFrame(list(result["Allocation"].items()), columns=['Facility-Location', 'Quantity'])

    print(tabulate(allocation_df.sort_values(by='Facility-Location'), headers='keys', tablefmt='grid'))

    print("\n")

Scenario 1: Original
Total Cost: $1318984.93
+----+--------------------------------+------------+
|    | Facility-Location              |   Quantity |
+====+================================+============+
|  9 | ('Cincinnati', 'Bridgeport')   |     111475 |
+----+--------------------------------+------------+
|  1 | ('Cincinnati', 'El Paso')      |       6800 |
+----+--------------------------------+------------+
|  6 | ('Cincinnati', 'Glendale')     |          0 |
+----+--------------------------------+------------+
|  3 | ('Cincinnati', 'Houston')      |     100447 |
+----+--------------------------------+------------+
|  7 | ('Cincinnati', 'Jacksonville') |      68486 |
+----+--------------------------------+------------+
|  4 | ('Cincinnati', 'Kansas City')  |      24570 |
+----+--------------------------------+------------+
|  8 | ('Cincinnati', 'Little Rock')  |     148586 |
+----+--------------------------------+------------+
|  5 | ('Cincinnati', 'Los Angeles')  |          0 |
+

In [ ]:
# Import necessary libraries
from pulp import *
import pandas as pd
from tabulate import tabulate

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [1000, 0.84, 0.83, 0.45, 0.36, 1000, 1000, 0.34, 0.34, 0.34, 1000],
    "Oakland": [0.22, 0.74, 0.49, 1000, 1000, 0.22, 0.22, 1000, 1000, 1000, 0.15]
}

# Capacities
max_cincinnati = 500000

# Define all sensitivity scenarios for production and freight cost adjustments (-10%, 0%, +10%)
cost_adjustments = [-0.10, 0.00, 0.10]  # -10%, 0%, +10%
scenario_combinations = []

# Generate all combinations of production and freight cost adjustments
for prod_adj in cost_adjustments:
    for freight_adj in cost_adjustments:
        scenario_combinations.append((prod_adj, freight_adj))

# Function to recalculate total cost and solve for a given adjustment
def run_sensitivity_scenario(prod_adjustment, freight_adjustment):
    # Adjust production costs
    adjusted_production_cost = {
        facility: cost * (1 + prod_adjustment) for facility, cost in production_cost.items()
    }

    # Solve the optimization problem with adjusted costs
    model = LpProblem("Minimize_Total_Cost_Sensitivity", LpMinimize)
    x = {
        (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
        for facility in ["Cincinnati", "Oakland"]
        for location in locations
    }

    # Objective function: Minimize total cost with adjusted production and freight costs
    model += lpSum(
        (adjusted_production_cost[facility] + (freight_cost[facility][i] * (1 + freight_adjustment) if freight_cost[facility][i] is not None else 0))
        * x[(facility, location)]
        for facility in ["Cincinnati", "Oakland"]
        for i, location in enumerate(locations)
        if freight_cost[facility][i] is not None
    )

    # Constraints: Demand must be satisfied for each location
    for i, location in enumerate(locations):
        model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

    # Capacity constraint for Cincinnati
    model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

    # Solve the adjusted problem
    model.solve()

    # Calculate total cost and extract allocation strategy
    total_cost = sum(
        x[(facility, location)].varValue *
        (adjusted_production_cost[facility] + (freight_cost[facility][locations.index(location)] * (1 + freight_adjustment) if freight_cost[facility][locations.index(location)] is not None else 0))
        for facility in ["Cincinnati", "Oakland"] for location in locations if x[(facility, location)].varValue > 0
    )
    allocation = [
        {
            "Location": location,
            "From Cincinnati": x[("Cincinnati", location)].varValue if x[("Cincinnati", location)].varValue > 0 else 0,
            "From Oakland": x[("Oakland", location)].varValue if x[("Oakland", location)].varValue > 0 else 0
        }
        for location in locations
    ]
    return total_cost, allocation

# Run all scenarios and collect results
sensitivity_results = []
for prod_adj, freight_adj in scenario_combinations:
    total_cost, allocation = run_sensitivity_scenario(prod_adj, freight_adj)
    scenario_label = f"Prod: {prod_adj * 100:.0f}%, Ship: {freight_adj * 100:.0f}%"
    sensitivity_results.append({
        "Scenario": scenario_label,
        "Total Cost": total_cost,
        "Allocation": allocation
    })

# Display detailed results for each scenario in separate lines with neat formatting
for result in sensitivity_results:
    print(f"Scenario: {result['Scenario']}")
    print(f"Total Cost: ${result['Total Cost']:.2f}")
    print("Allocation:")
    for allocation in result["Allocation"]:
        print(f"  {allocation['Location']}:")
        print(f"    Cincinnati: {allocation['From Cincinnati']} gallons")
        print(f"    Oakland: {allocation['From Oakland']} gallons")
    print("\n")

Scenario: Prod: -10%, Ship: -10%
Total Cost: $1187086.44
Allocation:
  Santa Ana:
    Cincinnati: 0 gallons
    Oakland: 22418.0 gallons
  El Paso:
    Cincinnati: 6800.0 gallons
    Oakland: 0 gallons
  Pendleton:
    Cincinnati: 39636.0 gallons
    Oakland: 40654.0 gallons
  Houston:
    Cincinnati: 100447.0 gallons
    Oakland: 0 gallons
  Kansas City:
    Cincinnati: 24570.0 gallons
    Oakland: 0 gallons
  Los Angeles:
    Cincinnati: 0 gallons
    Oakland: 64761.0 gallons
  Glendale:
    Cincinnati: 0 gallons
    Oakland: 33689.0 gallons
  Jacksonville:
    Cincinnati: 68486.0 gallons
    Oakland: 0 gallons
  Little Rock:
    Cincinnati: 148586.0 gallons
    Oakland: 0 gallons
  Bridgeport:
    Cincinnati: 111475.0 gallons
    Oakland: 0 gallons
  Sacramento:
    Cincinnati: 0 gallons
    Oakland: 112000.0 gallons


Scenario: Prod: -10%, Ship: 0%
Total Cost: $1213853.80
Allocation:
  Santa Ana:
    Cincinnati: 0 gallons
    Oakland: 22418.0 gallons
  El Paso:
    Cincinnati: 6800

In [ ]:
# Display detailed results for each scenario in a more tidy format using tabulate
for result in sensitivity_results:
    print(f"Scenario: {result['Scenario']}")
    print(f"Total Cost: ${result['Total Cost']:.2f}")

    allocation_df = pd.DataFrame(result["Allocation"])
    print(tabulate(allocation_df, headers='keys', tablefmt='grid'))

    print("\n")

Scenario: Prod: -10%, Ship: -10%
Total Cost: $1187086.44
+----+--------------+-------------------+----------------+
|    | Location     |   From Cincinnati |   From Oakland |
+====+==============+===================+================+
|  0 | Santa Ana    |                 0 |          22418 |
+----+--------------+-------------------+----------------+
|  1 | El Paso      |              6800 |              0 |
+----+--------------+-------------------+----------------+
|  2 | Pendleton    |             39636 |          40654 |
+----+--------------+-------------------+----------------+
|  3 | Houston      |            100447 |              0 |
+----+--------------+-------------------+----------------+
|  4 | Kansas City  |             24570 |              0 |
+----+--------------+-------------------+----------------+
|  5 | Los Angeles  |                 0 |          64761 |
+----+--------------+-------------------+----------------+
|  6 | Glendale     |                 0 |          33689 |

In [ ]:
from pulp import *
import pandas as pd
from tabulate import tabulate

# Data
locations = ["Santa Ana", "El Paso", "Pendleton", "Houston", "Kansas City",
             "Los Angeles", "Glendale", "Jacksonville", "Little Rock",
             "Bridgeport", "Sacramento"]
demand = [22418, 6800, 80290, 100447, 24570, 64761, 33689, 68486, 148586, 111475, 112000]

# Production costs
production_cost = {"Cincinnati": 1.20, "Oakland": 1.65}

# Freight costs
freight_cost = {
    "Cincinnati": [10000, 0.84, 0.83, 0.45, 0.36, 10000, 10000, 0.34, 0.34, 0.34, 10000],
    "Oakland": [0.22, 0.74, 0.49, 10000, 10000, 0.22, 0.22, 10000, 10000, 10000, 0.15]
}

# Capacities
max_cincinnati = 500000
max_oakland = float('inf')  # No cap on Oakland

# Define a very large number to represent prohibitively high costs
very_large_number = 1e6

def solve_optimization(production_cost, freight_cost):
    # Define the optimization problem
    model = LpProblem("Minimize_Total_Cost", LpMinimize)

    # Decision variables
    x = {
        (facility, location): LpVariable(f"x_{facility}_{location}", lowBound=0, cat="Continuous")
        for facility in ["Cincinnati", "Oakland"]
        for location in locations
    }

    # Objective function: Minimize total cost
    model += lpSum(
        (production_cost[facility] + (freight_cost[facility][i] if freight_cost[facility][i] is not None else very_large_number))
        * x[(facility, location)]
        for facility in ["Cincinnati", "Oakland"]
        for i, location in enumerate(locations)
        if freight_cost[facility][i] is not None
    )

    # Constraints: Demand must be satisfied for each location
    for i, location in enumerate(locations):
        model += lpSum(x[(facility, location)] for facility in ["Cincinnati", "Oakland"]) == demand[i]

    # Capacity constraint for Cincinnati
    model += lpSum(x[("Cincinnati", location)] for location in locations) <= max_cincinnati

    # Solve the problem
    model.solve()

    # Extract the results
    total_cost = value(model.objective)
    shipping_strategy = {(facility, location): x[(facility, location)].varValue for facility in ["Cincinnati", "Oakland"] for location in locations}

    return total_cost, shipping_strategy

# Original scenario
original_total_cost, original_shipping_strategy = solve_optimization(production_cost, freight_cost)

# Scenario with increased production costs by 10% for Cincinnati and decreased by 10% for Oakland
increased_production_cost_cincinnati = {k: v * (1.10 if k == "Cincinnati" else v) for k, v in production_cost.items()}
decreased_production_cost_oakland = {k: v * (0.90 if k == "Oakland" else v) for k, v in production_cost.items()}
increased_total_cost_cincinnati_decreased_oakland_production, increased_shipping_strategy_cincinnati_decreased_oakland_production = solve_optimization(increased_production_cost_cincinnati, freight_cost)

# Scenario with decreased production costs by 10% for Cincinnati and increased by 10% for Oakland
decreased_production_cost_cincinnati = {k: v * (0.90 if k == "Cincinnati" else v) for k, v in production_cost.items()}
increased_production_cost_oakland = {k: v * (1.10 if k == "Oakland" else v) for k, v in production_cost.items()}
decreased_total_cost_cincinnati_increased_oakland_production, decreased_shipping_strategy_cincinnati_increased_oakland_production = solve_optimization(decreased_production_cost_cincinnati, freight_cost)

# Scenario with increased freight costs by 10% for Cincinnati and decreased by 10% for Oakland
increased_freight_cost_cincinnati = {k: [v * (1.10 if k == "Cincinnati" else v) if v != very_large_number else v for v in vals] for k, vals in freight_cost.items()}
decreased_freight_cost_oakland = {k: [v * (0.90 if k == "Oakland" else v) if v != very_large_number else v for v in vals] for k, vals in freight_cost.items()}
increased_total_cost_cincinnati_decreased_oakland_freight, increased_shipping_strategy_cincinnati_decreased_oakland_freight = solve_optimization(production_cost, increased_freight_cost_cincinnati)

# Scenario with decreased freight costs by 10% for Cincinnati and increased by 10% for Oakland
decreased_freight_cost_cincinnati = {k: [v * (0.90 if k == "Cincinnati" else v) if v != very_large_number else v for v in vals] for k, vals in freight_cost.items()}
increased_freight_cost_oakland = {k: [v * (1.10 if k == "Oakland" else v) if v != very_large_number else v for v in vals] for k, vals in freight_cost.items()}
decreased_total_cost_cincinnati_increased_oakland_freight, decreased_shipping_strategy_cincinnati_increased_oakland_freight = solve_optimization(production_cost, decreased_freight_cost_cincinnati)

# Collect results into a list of dictionaries for better presentation
sensitivity_results = [
    {
        "Scenario": "Original",
        "Total Cost": original_total_cost,
        "Allocation": original_shipping_strategy
    },
    {
        "Scenario": "Increased Production Costs by 10% (Cincinnati) and Decreased by 10% (Oakland)",
        "Total Cost": increased_total_cost_cincinnati_decreased_oakland_production,
        "Allocation": increased_shipping_strategy_cincinnati_decreased_oakland_production
    },
    {
        "Scenario": "Decreased Production Costs by 10% (Cincinnati) and Increased by 10% (Oakland)",
        "Total Cost": decreased_total_cost_cincinnati_increased_oakland_production,
        "Allocation": decreased_shipping_strategy_cincinnati_increased_oakland_production
    },
    {
        "Scenario": "Increased Freight Costs by 10% (Cincinnati) and Decreased by 10% (Oakland)",
        "Total Cost": increased_total_cost_cincinnati_decreased_oakland_freight,
        "Allocation": increased_shipping_strategy_cincinnati_decreased_oakland_freight
    },
    {
        "Scenario": "Decreased Freight Costs by 10% (Cincinnati) and Increased by 10% (Oakland)",
        "Total Cost": decreased_total_cost_cincinnati_increased_oakland_freight,
        "Allocation": decreased_shipping_strategy_cincinnati_increased_oakland_freight
    }
]

# Display detailed results for each scenario in a more tidy format using tabulate and include scenario number
for idx, result in enumerate(sensitivity_results):
    print(f"Scenario {idx + 1}: {result['Scenario']}")
    print(f"Total Cost: ${result['Total Cost']:.2f}")

    allocation_df = pd.DataFrame(list(result["Allocation"].items()), columns=['Facility-Location', 'Quantity'])

    print(tabulate(allocation_df.sort_values(by='Facility-Location'), headers='keys', tablefmt='grid'))

    print("\n")

# Additional analysis: Capacity utilization
for idx, result in enumerate(sensitivity_results):
    allocation = result["Allocation"]
    cincinnati_utilization = sum(allocation[("Cincinnati", loc)] for loc in locations)
    oakland_utilization = sum(allocation[("Oakland", loc)] for loc in locations)
    print(f"Scenario {idx + 1}: {result['Scenario']}")
    print(f"Cincinnati Utilization: {cincinnati_utilization} / {max_cincinnati}")
    print(f"Oakland Utilization: {oakland_utilization} / 500000")
    print("\n")

Scenario 1: Original
Total Cost: $1318984.93
+----+--------------------------------+------------+
|    | Facility-Location              |   Quantity |
+====+================================+============+
|  9 | ('Cincinnati', 'Bridgeport')   |     111475 |
+----+--------------------------------+------------+
|  1 | ('Cincinnati', 'El Paso')      |       6800 |
+----+--------------------------------+------------+
|  6 | ('Cincinnati', 'Glendale')     |          0 |
+----+--------------------------------+------------+
|  3 | ('Cincinnati', 'Houston')      |     100447 |
+----+--------------------------------+------------+
|  7 | ('Cincinnati', 'Jacksonville') |      68486 |
+----+--------------------------------+------------+
|  4 | ('Cincinnati', 'Kansas City')  |      24570 |
+----+--------------------------------+------------+
|  8 | ('Cincinnati', 'Little Rock')  |     148586 |
+----+--------------------------------+------------+
|  5 | ('Cincinnati', 'Los Angeles')  |          0 |
+

##2e selected display

In [ ]:
# Collect results into a list of dictionaries for better presentation
sensitivity_results = [
    {
        "Scenario": "Original",
        "Total Cost": original_total_cost,
        "Allocation": original_shipping_strategy
    },
    {
        "Scenario": "Increased Production Costs by 10% (Cincinnati) and Decreased by 10% (Oakland)",
        "Total Cost": increased_total_cost_cincinnati_decreased_oakland_production,
        "Allocation": increased_shipping_strategy_cincinnati_decreased_oakland_production
    },
    {
        "Scenario": "Decreased Production Costs by 10% (Cincinnati) and Increased by 10% (Oakland)",
        "Total Cost": decreased_total_cost_cincinnati_increased_oakland_production,
        "Allocation": decreased_shipping_strategy_cincinnati_increased_oakland_production
    },
    {
        "Scenario": "Increased Freight Costs by 10% (Cincinnati) and Decreased by 10% (Oakland)",
        "Total Cost": increased_total_cost_cincinnati_decreased_oakland_freight,
        "Allocation": increased_shipping_strategy_cincinnati_decreased_oakland_freight
    },
    {
        "Scenario": "Decreased Freight Costs by 10% (Cincinnati) and Increased by 10% (Oakland)",
        "Total Cost": decreased_total_cost_cincinnati_increased_oakland_freight,
        "Allocation": decreased_shipping_strategy_cincinnati_increased_oakland_freight
    }
]

# Display detailed results for each scenario in a more tidy format using tabulate and remove the index column
for idx, result in enumerate(sensitivity_results):
    print(f"Scenario {idx + 1}: {result['Scenario']}")
    print(f"Total Cost: ${result['Total Cost']:.2f}")

    allocation_df = pd.DataFrame(list(result["Allocation"].items()), columns=['Facility-Location', 'Quantity'])

    print(tabulate(allocation_df[['Facility-Location', 'Quantity']].sort_values(by='Facility-Location').reset_index(drop=True), headers='keys', tablefmt='plain'))

    print("\n")

# Additional analysis: Capacity utilization
for idx, result in enumerate(sensitivity_results):
    allocation = result["Allocation"]
    cincinnati_utilization = sum(allocation[("Cincinnati", loc)] for loc in locations)
    oakland_utilization = sum(allocation[("Oakland", loc)] for loc in locations)
    print(f"Scenario {idx + 1}: {result['Scenario']}")
    print(f"Cincinnati Utilization: {cincinnati_utilization} / {max_cincinnati}")
    print(f"Oakland Utilization: {oakland_utilization} / 500000")
    print("\n")


Scenario 1: Original
Total Cost: $1318984.93
    Facility-Location                 Quantity
 0  ('Cincinnati', 'Bridgeport')        111475
 1  ('Cincinnati', 'El Paso')             6800
 2  ('Cincinnati', 'Glendale')               0
 3  ('Cincinnati', 'Houston')           100447
 4  ('Cincinnati', 'Jacksonville')       68486
 5  ('Cincinnati', 'Kansas City')        24570
 6  ('Cincinnati', 'Little Rock')       148586
 7  ('Cincinnati', 'Los Angeles')            0
 8  ('Cincinnati', 'Pendleton')          39636
 9  ('Cincinnati', 'Sacramento')             0
10  ('Cincinnati', 'Santa Ana')              0
11  ('Oakland', 'Bridgeport')                0
12  ('Oakland', 'El Paso')                   0
13  ('Oakland', 'Glendale')              33689
14  ('Oakland', 'Houston')                   0
15  ('Oakland', 'Jacksonville')              0
16  ('Oakland', 'Kansas City')               0
17  ('Oakland', 'Little Rock')               0
18  ('Oakland', 'Los Angeles')           64761
19  ('Oakland',

# Question 3 (complete)

##3a

In [ ]:
import pulp

# Define the problem
prob = pulp.LpProblem("Economic_Goals", pulp.LpMinimize)

# Define decision variables
G = pulp.LpVariable("G", lowBound=0, cat='Continuous')
T1 = pulp.LpVariable("T1", lowBound=0, cat='Continuous')
T2 = pulp.LpVariable("T2", lowBound=0, cat='Continuous')
C = pulp.LpVariable("C", lowBound=0, upBound=150, cat='Continuous')

# Define deviation variables for all goals
d1_pos = pulp.LpVariable("d1_pos", lowBound=0, cat='Continuous')
d1_neg = pulp.LpVariable("d1_neg", lowBound=0, cat='Continuous')
d2_pos = pulp.LpVariable("d2_pos", lowBound=0, cat='Continuous')
d2_neg = pulp.LpVariable("d2_neg", lowBound=0, cat='Continuous')
d3_pos = pulp.LpVariable("d3_pos", lowBound=0, cat='Continuous')
d3_neg = pulp.LpVariable("d3_neg", lowBound=0, cat='Continuous')
d4_pos = pulp.LpVariable("d4_pos", lowBound=0, cat='Continuous')
d4_neg = pulp.LpVariable("d4_neg", lowBound=0, cat='Continuous')

# Step 1: Optimize the Highest Priority Goal
# Add objective function for the highest priority goal (minimize negative deviation from Goal 1)
prob += d1_neg, "Objective"
# Add constraints for Goal 1 (Balance the budget)
prob += 1.5*G + 25*T1 + 15*T2 + C - d1_pos + d1_neg == 1000
# Add constraint to ensure T2 >= T1
prob += T2 >= T1
# Solve the problem for the highest priority goal
prob.solve()
# Store the value of d1_neg after solving the problem
d1_neg_value = d1_neg.varValue

# Step 2: Optimize the Second Priority Goal
# Add objective function for the second priority goal (minimize positive deviation from Goal 2)
prob += d2_pos, "Objective"
# Add constraints for Goal 2 (Cut spending by at most $150 billion)
prob += C - d2_pos + d2_neg == 150
# Fix the deviation variable for Goal 1 to its value from Step 1
prob += d1_neg == d1_neg_value
# Solve the problem for the second priority goal
prob.solve()
# Store the value of d2_pos after solving the problem
d2_pos_value = d2_pos.varValue

# Step 3: Optimize the Third Priority Goal
# Add objective function for the third priority goal (minimize positive deviation from Goal 3)
prob += d3_pos, "Objective"
# Add constraints for Goal 3 (Raise at most $550 billion in taxes from the upper class)
prob += 0.5*G + 5*T1 + 15*T2 - d3_pos + d3_neg == 550
# Fix the deviation variables for Goals 1 and 2 to their values from previous steps
prob += d2_pos == d2_pos_value
# Solve the problem for the third priority goal
prob.solve()
# Store the value of d3_pos after solving the problem
d3_pos_value = d3_pos.varValue

# Step 4: Optimize the Fourth Priority Goal
# Add objective function for the fourth priority goal (minimize positive deviation from Goal 4)
prob += d4_pos, "Objective"
# Add constraints for Goal 4 (Raise at most $350 billion in taxes from the lower class)
prob += G + 20*T1 - d4_pos + d4_neg == 350
# Fix the deviation variables for Goals 1, 2, and 3 to their values from previous steps
prob += d3_pos == d3_pos_value
# Solve the problem for the fourth priority goal
prob.solve()

# Print final results
print(f"Model Status after Goal 4: {pulp.LpStatus[prob.status]}")
print(f"G (Gas tax rate): {G.varValue} cents")
print(f"T1 (Tax rate on first $30,000): {T1.varValue}")
print(f"T2 (Tax rate on income over $30,000): {T2.varValue}")
print(f"C (Spending cut): {C.varValue} billion dollars")
print(f"Deviation from Goal 1: Positive {d1_pos.varValue}, Negative {d1_neg.varValue}")
print(f"Deviation from Goal 2: Positive {d2_pos.varValue}, Negative {d2_neg.varValue}")
print(f"Deviation from Goal 3: Positive {d3_pos.varValue}, Negative {d3_neg.varValue}")
print(f"Deviation from Goal 4: Positive {d4_pos.varValue}, Negative {d4_neg.varValue}")

Model Status after Goal 4: Optimal
G (Gas tax rate): 300.0 cents
T1 (Tax rate on first $30,000): 0.0
T2 (Tax rate on income over $30,000): 26.666667
C (Spending cut): 150.0 billion dollars
Deviation from Goal 1: Positive 0.0, Negative 0.0
Deviation from Goal 2: Positive 0.0, Negative 0.0
Deviation from Goal 3: Positive 0.0, Negative 0.0
Deviation from Goal 4: Positive 0.0, Negative 50.0


##3b


In [ ]:
import numpy as np

# Define the sensitivity analysis function
def sensitivity_analysis(G, T1, T2):
    # Define the range of changes for each variable
    delta = np.linspace(-0.1, 0.1, 21)

    # Initialize lists to store results
    revenue_G = []
    revenue_T1 = []
    revenue_T2 = []

    # Calculate revenue for changes in G
    for d in delta:
        new_G = G * (1 + d)
        revenue = 1.5 * new_G + 25 * T1 + 15 * T2
        revenue_G.append(revenue)

    # Calculate revenue for changes in T1
    for d in delta:
        new_T1 = T1 * (1 + d)
        revenue = 1.5 * G + 25 * new_T1 + 15 * T2
        revenue_T1.append(revenue)

    # Calculate revenue for changes in T2
    for d in delta:
        new_T2 = T2 * (1 + d)
        revenue = 1.5 * G + 25 * T1 + 15 * new_T2
        revenue_T2.append(revenue)

    return delta, revenue_G, revenue_T1, revenue_T2

# Optimal values from the previous solution
G_opt = 350
T1_opt = 0
T2_opt = 21.66667

# Conduct sensitivity analysis
delta, revenue_G, revenue_T1, revenue_T2 = sensitivity_analysis(G_opt, T1_opt, T2_opt)

# Print the results
print("Sensitivity Analysis Results:")
print("Changes in G:")
for d, rev in zip(delta, revenue_G):
    print(f"Change: {d*100:.1f}%, Revenue: {rev:.2f} billion dollars")

print("\nChanges in T1:")
for d, rev in zip(delta, revenue_T1):
    print(f"Change: {d*100:.1f}%, Revenue: {rev:.2f} billion dollars")

print("\nChanges in T2:")
for d, rev in zip(delta, revenue_T2):
    print(f"Change: {d*100:.1f}%, Revenue: {rev:.2f} billion dollars")

# Determine which variable has the most significant impact on total revenue
impact_G = max(revenue_G) - min(revenue_G)
impact_T1 = max(revenue_T1) - min(revenue_T1)
impact_T2 = max(revenue_T2) - min(revenue_T2)

most_significant_impact = max(impact_G, impact_T1, impact_T2)
if most_significant_impact == impact_G:
    most_significant_variable = "G"
elif most_significant_impact == impact_T1:
    most_significant_variable = "T1"
else:
    most_significant_variable = "T2"

print(f"\nThe variable with the most significant impact on total revenue is: {most_significant_variable}")


Sensitivity Analysis Results:
Changes in G:
Change: -10.0%, Revenue: 797.50 billion dollars
Change: -9.0%, Revenue: 802.75 billion dollars
Change: -8.0%, Revenue: 808.00 billion dollars
Change: -7.0%, Revenue: 813.25 billion dollars
Change: -6.0%, Revenue: 818.50 billion dollars
Change: -5.0%, Revenue: 823.75 billion dollars
Change: -4.0%, Revenue: 829.00 billion dollars
Change: -3.0%, Revenue: 834.25 billion dollars
Change: -2.0%, Revenue: 839.50 billion dollars
Change: -1.0%, Revenue: 844.75 billion dollars
Change: 0.0%, Revenue: 850.00 billion dollars
Change: 1.0%, Revenue: 855.25 billion dollars
Change: 2.0%, Revenue: 860.50 billion dollars
Change: 3.0%, Revenue: 865.75 billion dollars
Change: 4.0%, Revenue: 871.00 billion dollars
Change: 5.0%, Revenue: 876.25 billion dollars
Change: 6.0%, Revenue: 881.50 billion dollars
Change: 7.0%, Revenue: 886.75 billion dollars
Change: 8.0%, Revenue: 892.00 billion dollars
Change: 9.0%, Revenue: 897.25 billion dollars
Change: 10.0%, Revenue: 9

# Question 4 (complete)

##4a

In [ ]:
import numpy as np
import pulp

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5], # Production A
    [6, 2, 4], # Production B
    [5, 4, 3], # Production C
    [7, 5, 6]  # Production D
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7], # Warehouse 1
    [1, 4, 2, 3, 5, 6], # Warehouse 2
    [5, 3, 4, 2, 3, 4]  # Warehouse 3
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc.shape[0]

# Create the linear programming problem
prob = pulp.LpProblem("Minimise_Transportation_Cost", pulp.LpMinimize)

# Decision variables for flows from production centres to warehouses
flow_pc_wh = pulp.LpVariable.dicts("Flow_PC_WH", ((i, j) for i in range(num_pc) for j in range(num_wh)), lowBound=0, cat='Integer')

# Decision variables for flows from warehouses to distribution centres
flow_wh_dc = pulp.LpVariable.dicts("Flow_WH_DC", ((j, k) for j in range(num_wh) for k in range(num_dc)), lowBound=0, cat='Integer')

# Objective function: Minimize total transportation cost
prob += pulp.lpSum(cost_pc_wh[i][j] * flow_pc_wh[(i, j)] for i in range(num_pc) for j in range(num_wh)) + \
        pulp.lpSum(cost_wh_dc[j][k] * flow_wh_dc[(j, k)] for j in range(num_wh) for k in range(num_dc))

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for j in range(num_wh)) == supply_pc[i], f"Supply_Constraint_PC_{i}"

# Demand constraints for warehouses to distribution centres
for k in range(num_dc):
    prob += pulp.lpSum(flow_wh_dc[(j, k)] for j in range(num_wh)) == demand_dc[k], f"Demand_Constraint_DC_{k}"

# Flow conservation constraints for warehouses
for j in range(num_wh):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for i in range(num_pc)) == pulp.lpSum(flow_wh_dc[(j, k)] for k in range(num_dc)), f"Flow_Conservation_WH_{j}"

# Solve the problem
prob.solve()

print("Optimal flows from  production to warehouses:")
for i in range(num_pc):
    for j in range(num_wh):
        print(f"Flow from Production {i} to Warehouse {j}: {pulp.value(flow_pc_wh[(i, j)])}")

print("Optimal flows from warehouses to distribution centres:")
for j in range(num_wh):
    for k in range(num_dc):
        print(f"Flow from Warehouse {j} to Distribution Centre {k}: {pulp.value(flow_wh_dc[(j, k)])}")

print("Optimal transportation cost:")
print(pulp.value(prob.objective))

Optimal flows from  production to warehouses:
Flow from Production 0 to Warehouse 0: 40.0
Flow from Production 0 to Warehouse 1: 30.0
Flow from Production 0 to Warehouse 2: 0.0
Flow from Production 1 to Warehouse 0: 0.0
Flow from Production 1 to Warehouse 1: 90.0
Flow from Production 1 to Warehouse 2: 0.0
Flow from Production 2 to Warehouse 0: 0.0
Flow from Production 2 to Warehouse 1: 0.0
Flow from Production 2 to Warehouse 2: 80.0
Flow from Production 3 to Warehouse 0: 0.0
Flow from Production 3 to Warehouse 1: 60.0
Flow from Production 3 to Warehouse 2: 0.0
Optimal flows from warehouses to distribution centres:
Flow from Warehouse 0 to Distribution Centre 0: 0.0
Flow from Warehouse 0 to Distribution Centre 1: 40.0
Flow from Warehouse 0 to Distribution Centre 2: 0.0
Flow from Warehouse 0 to Distribution Centre 3: 0.0
Flow from Warehouse 0 to Distribution Centre 4: 0.0
Flow from Warehouse 0 to Distribution Centre 5: 0.0
Flow from Warehouse 1 to Distribution Centre 0: 50.0
Flow from Wa

##4b

In [ ]:
import numpy as np
import pulp

#Define the transportation costs from production centres to warehouses with increased costs for Warehouse 2 (middle)
cost_pc_wh = np.array([
    [4, 4, 5],  # Increased cost by £1
    [6, 3, 4],  # Increased cost by £1
    [5, 5, 3],  # Increased cost by £1
    [7, 6, 6]   # Increased cost by £1
])

# Define the transportation costs from warehouses to distribution centres
# Increase the costs for Warehouse 2 by £1 per unit
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7], # Warehouse 1
    [2, 5, 3, 4, 6, 7], # Warehouse 2 with increased costs
    [5, 3, 4, 2, 3, 4]  # Warehouse 3
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc.shape[0]

# Create the linear programming problem
prob = pulp.LpProblem("Minimise_Transportation_Cost", pulp.LpMinimize)

# Decision variables for flows from production centres to warehouses
flow_pc_wh = pulp.LpVariable.dicts("Flow_PC_WH", ((i, j) for i in range(num_pc) for j in range(num_wh)), lowBound=0, cat='Integer')

# Decision variables for flows from warehouses to distribution centres
flow_wh_dc = pulp.LpVariable.dicts("Flow_WH_DC", ((j, k) for j in range(num_wh) for k in range(num_dc)), lowBound=0, cat='Integer')

# Objective function: Minimize total transportation cost
prob += pulp.lpSum(cost_pc_wh[i][j] * flow_pc_wh[(i, j)] for i in range(num_pc) for j in range(num_wh)) + \
        pulp.lpSum(cost_wh_dc[j][k] * flow_wh_dc[(j, k)] for j in range(num_wh) for k in range(num_dc))

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for j in range(num_wh)) == supply_pc[i], f"Supply_Constraint_PC_{i}"

# Demand constraints for warehouses to distribution centres
for k in range(num_dc):
    prob += pulp.lpSum(flow_wh_dc[(j, k)] for j in range(num_wh)) == demand_dc[k], f"Demand_Constraint_DC_{k}"

# Flow conservation constraints for warehouses
for j in range(num_wh):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for i in range(num_pc)) == pulp.lpSum(flow_wh_dc[(j, k)] for k in range(num_dc)), f"Flow_Conservation_WH_{j}"

# Solve the problem
prob.solve()

print("Optimal flows from production centres to warehouses:")
for i in range(num_pc):
    for j in range(num_wh):
        print(f"Flow from Production {i} to Warehouse {j}: {pulp.value(flow_pc_wh[(i, j)])}")

print("Optimal flows from warehouses to distribution centres:")
for j in range(num_wh):
    for k in range(num_dc):
        print(f"Flow from Warehouse {j} to Distribution Centre {k}: {pulp.value(flow_wh_dc[(j, k)])}")

print("Optimal transportation cost:")
print(pulp.value(prob.objective))

Optimal flows from production centres to warehouses:
Flow from Production 0 to Warehouse 0: 40.0
Flow from Production 0 to Warehouse 1: 30.0
Flow from Production 0 to Warehouse 2: 0.0
Flow from Production 1 to Warehouse 0: 0.0
Flow from Production 1 to Warehouse 1: 80.0
Flow from Production 1 to Warehouse 2: 10.0
Flow from Production 2 to Warehouse 0: 0.0
Flow from Production 2 to Warehouse 1: 0.0
Flow from Production 2 to Warehouse 2: 80.0
Flow from Production 3 to Warehouse 0: 0.0
Flow from Production 3 to Warehouse 1: 0.0
Flow from Production 3 to Warehouse 2: 60.0
Optimal flows from warehouses to distribution centres:
Flow from Warehouse 0 to Distribution Centre 0: 0.0
Flow from Warehouse 0 to Distribution Centre 1: 40.0
Flow from Warehouse 0 to Distribution Centre 2: 0.0
Flow from Warehouse 0 to Distribution Centre 3: 0.0
Flow from Warehouse 0 to Distribution Centre 4: 0.0
Flow from Warehouse 0 to Distribution Centre 5: 0.0
Flow from Warehouse 1 to Distribution Centre 0: 50.0
Flow

##4c

In [ ]:
import numpy as np
import pulp

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5], # Production A
    [6, 2, 4], # Production B
    [5, 4, 3], # Production C
    [7, 5, 6]  # Production D
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 6, 7], # Warehouse 1
    [1, 4, 2, 3, 5, 6], # Warehouse 2
    [5, 3, 4, 2, 3, 4]  # Warehouse 3
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc.shape[0]

# Create the linear programming problem
prob = pulp.LpProblem("Minimise_Transportation_Cost", pulp.LpMinimize)

# Decision variables for flows from production centres to warehouses
flow_pc_wh = pulp.LpVariable.dicts("Flow_PC_WH", ((i, j) for i in range(num_pc) for j in range(num_wh)), lowBound=0, cat='Integer')

# Decision variables for flows from warehouses to distribution centres
flow_wh_dc = pulp.LpVariable.dicts("Flow_WH_DC", ((j, k) for j in range(num_wh) for k in range(num_dc)), lowBound=0, cat='Integer')

# Objective function: Minimize total transportation cost
prob += pulp.lpSum(cost_pc_wh[i][j] * flow_pc_wh[(i, j)] for i in range(num_pc) for j in range(num_wh)) + \
        pulp.lpSum(cost_wh_dc[j][k] * flow_wh_dc[(j, k)] for j in range(num_wh) for k in range(num_dc))

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for j in range(num_wh)) == supply_pc[i], f"Supply_Constraint_PC_{i}"

# Demand constraints for warehouses to distribution centres
for k in range(num_dc):
    prob += pulp.lpSum(flow_wh_dc[(j, k)] for j in range(num_wh)) == demand_dc[k], f"Demand_Constraint_DC_{k}"

# Flow conservation constraints for warehouses
for j in range(num_wh):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for i in range(num_pc)) == pulp.lpSum(flow_wh_dc[(j, k)] for k in range(num_dc)), f"Flow_Conservation_WH_{j}"

# Handling capacity constraint for Warehouse 1 (not to exceed 50 units)
prob += pulp.lpSum(flow_pc_wh[(i, 0)] for i in range(num_pc)) <= 50, "Capacity_Constraint_WH_1"

# Solve the problem
prob.solve()

print("Optimal flows from production centres to warehouses:")
for i in range(num_pc):
    for j in range(num_wh):
        print(f"Flow from Production {i} to Warehouse {j}: {pulp.value(flow_pc_wh[(i, j)])}")

print("Optimal flows from warehouses to distribution centres:")
for j in range(num_wh):
    for k in range(num_dc):
        print(f"Flow from Warehouse {j} to Distribution Centre {k}: {pulp.value(flow_wh_dc[(j, k)])}")

print("Optimal transportation cost:")
print(pulp.value(prob.objective))

Optimal flows from production centres to warehouses:
Flow from Production 0 to Warehouse 0: 40.0
Flow from Production 0 to Warehouse 1: 30.0
Flow from Production 0 to Warehouse 2: 0.0
Flow from Production 1 to Warehouse 0: 0.0
Flow from Production 1 to Warehouse 1: 90.0
Flow from Production 1 to Warehouse 2: 0.0
Flow from Production 2 to Warehouse 0: 0.0
Flow from Production 2 to Warehouse 1: 0.0
Flow from Production 2 to Warehouse 2: 80.0
Flow from Production 3 to Warehouse 0: 0.0
Flow from Production 3 to Warehouse 1: 60.0
Flow from Production 3 to Warehouse 2: 0.0
Optimal flows from warehouses to distribution centres:
Flow from Warehouse 0 to Distribution Centre 0: 0.0
Flow from Warehouse 0 to Distribution Centre 1: 40.0
Flow from Warehouse 0 to Distribution Centre 2: 0.0
Flow from Warehouse 0 to Distribution Centre 3: 0.0
Flow from Warehouse 0 to Distribution Centre 4: 0.0
Flow from Warehouse 0 to Distribution Centre 5: 0.0
Flow from Warehouse 1 to Distribution Centre 0: 50.0
Flow 

##4d

In [ ]:
import numpy as np
import pulp

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5], # Production A
    [6, 2, 4], # Production B
    [5, 4, 3], # Production C
    [7, 5, 6]  # Production D
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [2, 5, 4, 6, 7], # Warehouse 1
    [4, 2, 3, 5, 6], # Warehouse 2
    [3, 4, 2, 3, 4]  # Warehouse 3
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Remove DC5 and equally assign its demand to others
demand_dc_new = np.array([40 + 10, 60 + 10, 70 + 10, 50 + 10, 30 + 10])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc_new.shape[0]  # Update this to use the new demand

# Create the linear programming problem
prob = pulp.LpProblem("Minimise_Transportation_Cost", pulp.LpMinimize)

# Decision variables for flows from production centres to warehouses
flow_pc_wh = pulp.LpVariable.dicts("Flow_PC_WH", ((i, j) for i in range(num_pc) for j in range(num_wh)), lowBound=0, cat='Integer')

# Decision variables for flows from warehouses to distribution centres
flow_wh_dc = pulp.LpVariable.dicts("Flow_WH_DC", ((j, k) for j in range(num_wh) for k in range(num_dc)), lowBound=0, cat='Integer')

# Objective function: Minimize total transportation cost
prob += pulp.lpSum(cost_pc_wh[i][j] * flow_pc_wh[(i, j)] for i in range(num_pc) for j in range(num_wh)) + \
        pulp.lpSum(cost_wh_dc[j][k] * flow_wh_dc[(j, k)] for j in range(num_wh) for k in range(num_dc))

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for j in range(num_wh)) == supply_pc[i], f"Supply_Constraint_PC_{i}"

# Demand constraints for warehouses to distribution centres
for k in range(num_dc):
    prob += pulp.lpSum(flow_wh_dc[(j, k)] for j in range(num_wh)) == demand_dc_new[k], f"Demand_Constraint_DC_{k}"  # Use new demand

# Flow conservation constraints for warehouses
for j in range(num_wh):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for i in range(num_pc)) == pulp.lpSum(flow_wh_dc[(j, k)] for k in range(num_dc)), f"Flow_Conservation_WH_{j}"

# Solve the problem
prob.solve()

print("Optimal flows from production to warehouses:")
for i in range(num_pc):
    for j in range(num_wh):
        print(f"Flow from Production {i} to Warehouse {j}: {pulp.value(flow_pc_wh[(i, j)])}")

print("Optimal flows from warehouses to distribution centres:")
for j in range(num_wh):
    for k in range(num_dc):
        print(f"Flow from Warehouse {j} to Distribution Centre {k}: {pulp.value(flow_wh_dc[(j, k)])}")

print("Optimal transportation cost by removing DC1:")
print(pulp.value(prob.objective))

Optimal flows from production to warehouses:
Flow from Production 0 to Warehouse 0: 50.0
Flow from Production 0 to Warehouse 1: 20.0
Flow from Production 0 to Warehouse 2: 0.0
Flow from Production 1 to Warehouse 0: 0.0
Flow from Production 1 to Warehouse 1: 90.0
Flow from Production 1 to Warehouse 2: 0.0
Flow from Production 2 to Warehouse 0: 0.0
Flow from Production 2 to Warehouse 1: 0.0
Flow from Production 2 to Warehouse 2: 80.0
Flow from Production 3 to Warehouse 0: 0.0
Flow from Production 3 to Warehouse 1: 0.0
Flow from Production 3 to Warehouse 2: 60.0
Optimal flows from warehouses to distribution centres:
Flow from Warehouse 0 to Distribution Centre 0: 50.0
Flow from Warehouse 0 to Distribution Centre 1: 0.0
Flow from Warehouse 0 to Distribution Centre 2: 0.0
Flow from Warehouse 0 to Distribution Centre 3: 0.0
Flow from Warehouse 0 to Distribution Centre 4: 0.0
Flow from Warehouse 1 to Distribution Centre 0: 0.0
Flow from Warehouse 1 to Distribution Centre 1: 70.0
Flow from War

In [ ]:
import numpy as np
import pulp

# Define the transportation costs from production centres to warehouses
cost_pc_wh = np.array([
    [4, 3, 5], # Production A
    [6, 2, 4], # Production B
    [5, 4, 3], # Production C
    [7, 5, 6]  # Production D
])

# Define the transportation costs from warehouses to distribution centres
cost_wh_dc = np.array([
    [3, 2, 5, 4, 7], # Warehouse 1
    [1, 4, 2, 3, 6], # Warehouse 2
    [5, 3, 4, 2, 4]  # Warehouse 3
])

# Define the supply at each production centre
supply_pc = np.array([70, 90, 80, 60])

# Define the demand at each distribution centre
demand_dc = np.array([50, 40, 60, 70, 50, 30])

# Remove DC5 and equally assign its demand to others
demand_dc_new = np.array([50 + 10, 40 + 10, 60 + 10, 70 + 10, 30 + 10])

# Number of production centres, warehouses and distribution centres
num_pc = len(supply_pc)
num_wh = cost_pc_wh.shape[1]
num_dc = demand_dc_new.shape[0]  # Update this to use the new demand

# Create the linear programming problem
prob = pulp.LpProblem("Minimise_Transportation_Cost", pulp.LpMinimize)

# Decision variables for flows from production centres to warehouses
flow_pc_wh = pulp.LpVariable.dicts("Flow_PC_WH", ((i, j) for i in range(num_pc) for j in range(num_wh)), lowBound=0, cat='Integer')

# Decision variables for flows from warehouses to distribution centres
flow_wh_dc = pulp.LpVariable.dicts("Flow_WH_DC", ((j, k) for j in range(num_wh) for k in range(num_dc)), lowBound=0, cat='Integer')

# Objective function: Minimize total transportation cost
prob += pulp.lpSum(cost_pc_wh[i][j] * flow_pc_wh[(i, j)] for i in range(num_pc) for j in range(num_wh)) + \
        pulp.lpSum(cost_wh_dc[j][k] * flow_wh_dc[(j, k)] for j in range(num_wh) for k in range(num_dc))

# Supply constraints for production centres to warehouses
for i in range(num_pc):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for j in range(num_wh)) == supply_pc[i], f"Supply_Constraint_PC_{i}"

# Demand constraints for warehouses to distribution centres
for k in range(num_dc):
    prob += pulp.lpSum(flow_wh_dc[(j, k)] for j in range(num_wh)) == demand_dc_new[k], f"Demand_Constraint_DC_{k}"  # Use new demand

# Flow conservation constraints for warehouses
for j in range(num_wh):
    prob += pulp.lpSum(flow_pc_wh[(i, j)] for i in range(num_pc)) == pulp.lpSum(flow_wh_dc[(j, k)] for k in range(num_dc)), f"Flow_Conservation_WH_{j}"

# Solve the problem
prob.solve()

print("Optimal flows from production to warehouses:")
for i in range(num_pc):
    for j in range(num_wh):
        print(f"Flow from Production {i} to Warehouse {j}: {pulp.value(flow_pc_wh[(i, j)])}")

print("Optimal flows from warehouses to distribution centres:")
for j in range(num_wh):
    for k in range(num_dc):
        print(f"Flow from Warehouse {j} to Distribution Centre {k}: {pulp.value(flow_wh_dc[(j, k)])}")

print("Optimal transportation cost by removing DC5:")
print(pulp.value(prob.objective))

Optimal flows from production to warehouses:
Flow from Production 0 to Warehouse 0: 50.0
Flow from Production 0 to Warehouse 1: 20.0
Flow from Production 0 to Warehouse 2: 0.0
Flow from Production 1 to Warehouse 0: 0.0
Flow from Production 1 to Warehouse 1: 90.0
Flow from Production 1 to Warehouse 2: 0.0
Flow from Production 2 to Warehouse 0: 0.0
Flow from Production 2 to Warehouse 1: 0.0
Flow from Production 2 to Warehouse 2: 80.0
Flow from Production 3 to Warehouse 0: 0.0
Flow from Production 3 to Warehouse 1: 60.0
Flow from Production 3 to Warehouse 2: 0.0
Optimal flows from warehouses to distribution centres:
Flow from Warehouse 0 to Distribution Centre 0: 0.0
Flow from Warehouse 0 to Distribution Centre 1: 50.0
Flow from Warehouse 0 to Distribution Centre 2: 0.0
Flow from Warehouse 0 to Distribution Centre 3: 0.0
Flow from Warehouse 0 to Distribution Centre 4: 0.0
Flow from Warehouse 1 to Distribution Centre 0: 60.0
Flow from Warehouse 1 to Distribution Centre 1: 0.0
Flow from War